In [34]:
import sys
import os
sys.path.append('/g/data/jk72/ll6859/ACCESS_aero_eval/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
import scipy.special
from datetime import datetime

import dask
import warnings

In [35]:
from dask.distributed import Client
client = Client()  # set up local cluster
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/45377/status,
Dashboard: /proxy/45377/status,Workers: 4
Total threads: 8,Total memory: 22.46 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39203,Workers: 4
Dashboard: /proxy/45377/status,Total threads: 8
Started: Just now,Total memory: 22.46 GiB
Comm: tcp://127.0.0.1:42067,Total threads: 2
Dashboard: /proxy/40069/status,Memory: 5.62 GiB
Nanny: tcp://127.0.0.1:34433,


In [36]:
dask.config.set(**{'array.slicing.split_large_chunks': True});

In [37]:
plt.style.use('default')
warnings.filterwarnings('ignore')

In [38]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

# RVInvestigator

In [39]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/RVInvestigator/'
aer = pd.read_csv(f_path+'rvigaw_cn10_2016to2019_L2.csv', index_col='datetime')
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn10_avg','cn10_std','latitude','longitude']].dropna()
aer.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)

met = pd.read_csv(f_path+'RVI-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']>60]=np.nan
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2020_05_29d_RVI_1420_Radon.xlsx', sheet_name='RVI1420', index_col='DateTime')
rdn['time'] = pd.DatetimeIndex(rdn.index)
rdn.index = rdn['time']
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]','RnSTP']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std', 'RnSTP': 'radon_stp'}, inplace=True)

In [40]:
# convert observations to a dataset
rvi_ds = aer.to_xarray()
rvi_ds = rvi_ds.assign(met)
rvi_ds = rvi_ds.assign(rdn)

In [41]:
# remove outliers
rvi_ds = rvi_ds.where(rvi_ds['cn10_avg']<10000)

In [42]:
# Edit data attributes
rvi_ds.attrs['title'] = 'Aerosol, meteorology and radon observations for the 2016-2019 RV Investigator voyage.'
rvi_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [43]:
# add new source coordinate
rvi_ds = rvi_ds.assign_coords(Source='RVI')

In [44]:
rvi_ds.load().to_netcdf(path=f_path+'rvi1619_obs.nc')

# Cold Water Trial

bx400

In [45]:
# f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/ColdWaterTrial/'

# df = pd.read_csv(f_path+'CWT_ProcessedAerosolData_1min.csv', delimiter=',', index_col='date')
# df['time'] = pd.DatetimeIndex(df.index)
# df.index = df['time'].astype('datetime64[ns]')
# df = df[['lat','lon','AirTemp_port','AirTemp_stbd','atmPress','WindSpdTru','Radon_hrly','CN_mean']]
# df = df.dropna()
# df = df.resample('1D', kind='time').mean()

# Reef 2 Rainforest

In [46]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Reef2Rain/'

df = pd.read_csv(f_path+'R2R_RV_aerosol.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df = df[['lat','lon','air_temp','atmPress','WindDir', 'wind_spd', 'Radon_hrly','CN3_mean','CN10_mean','CCN_mean']]
df.rename(columns={'atmPress': 'air_pressure', 
                   'WindDir': 'wind_dir', 
                   'Radon_hrly': 'radon', 
                   'CN3_mean': 'cn3_avg', 
                   'CN10_mean': 'cn10_avg', 
                   'CCN_mean': 'ccn_avg'}, inplace=True)

In [47]:
# convert observations to a dataset
r2r_ds = df.to_xarray()

In [48]:
# remove outliers
r2r_ds = r2r_ds.where(r2r_ds['cn10_avg']<10000)

# Edit data attributes
r2r_ds.attrs['title'] = 'Aerosol and meteorology observations for the 2016 Reef 2 Rainforest voyage.'
r2r_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [49]:
# add new source coordinate
r2r_ds = r2r_ds.assign_coords(Source='Reef2Rain')

In [50]:
r2r_ds.load().to_netcdf(path=f_path+'r2r16_obs.nc')

# Ice 2 Equator

In [51]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Ice2Equator/'

df = pd.read_csv(f_path+'I2E_ProcessedAerosolData_1min.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df = df[['lat','lon','air_temp','atmPress','wind_spd','WindDirRel_vmean','Radon_hrly','CN_mean','CN_std']]
df.rename(columns={'atmPress': 'air_pressure',
                   'WindDirRel_vmean':'wind_dir',
                   'Radon_hrly': 'radon', 
                   'CN_mean': 'cn3_avg', 
                   'CN_std': 'cn3_std'}, inplace=True)

In [52]:
# convert observations to a dataset
i2e_ds = df.to_xarray()

In [53]:
# remove outliers
i2e_ds = i2e_ds.where(i2e_ds['cn3_avg']<10000)

# Edit data attributes
i2e_ds.attrs['title'] = 'Aerosol and meteorology observations for the 2016 Ice 2 Equator voyage.'
i2e_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [54]:
# add new source coordinate
i2e_ds = i2e_ds.assign_coords(Source='Ice2Equ')

In [55]:
i2e_ds.load().to_netcdf(path=f_path+'i2e16_obs.nc')

# CAPRICORN I

In [56]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Capricorn/'

aer = pd.read_hdf(f_path+'aerosol_1sec.h5')
aer.index.name = 'time'
aer = aer.resample('1D', kind='time').mean()
aer.rename(columns={'ccn_0.5504': 'ccn_avg',
                   'ccn_sigma': 'ccn_std', 
                   'cn3': 'cn3_avg', 
                   'cn3_sigma': 'cn3_std'}, inplace=True)

met = pd.read_csv(f_path+'in2016_v02uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
met = met.resample('1D').mean()
met['air_temp'] = (met['portAirTemp(degC)']+met['stbdAirTemp(degC)'])/2
met = met[['latitude(degree_north)','longitude(degree_east)','air_temp','atmPressure(mbar)','rain(mm)']]
met.rename(columns = {"latitude(degree_north)": "lat",
                      "longitude(degree_east)": "lon",
                      "atmPressure(mbar)": "air_pressure",
                      "rain(mm)": "total_rain"}, inplace=True)
met.index.name = 'time'   

In [57]:
# convert observations to a dataset
cap1_ds = aer.to_xarray()
cap1_ds = cap1_ds.assign(met)

In [58]:
# remove outliers
cap1_ds = cap1_ds.where(cap1_ds['cn3_avg']<10000)

# Edit data attributes
cap1_ds.attrs['title'] = 'Aerosol and meteorology observations for the 2016 CAPRICORN I voyage.'
cap1_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [59]:
# add new source coordinate
cap1_ds = cap1_ds.assign_coords(Source='Cap1')

In [60]:
cap1_ds.load().to_netcdf(path=f_path+'cap16_obs.nc')

# CAPRICORN II

In [61]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Capricorn/'

aer = pd.read_csv(f_path+'capricorn2_cpc3772_L2_1H.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time'].astype('datetime64[ns]')
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn_avg', 'cn_std']]
aer.rename(columns={'cn_avg': 'cn10_avg',
                   'cn_std': 'cn10_std'}, inplace=True)

met = pd.read_csv(f_path+'in2018_v01uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
for v in met.select_dtypes('object'):
    met[v] = pd.to_numeric(met[v], errors='coerce')
met.index.name = 'time'
met = met.resample('1D').mean()
met['air_temp'] = (met['portAirTemp(degC)']+met['stbdAirTemp(degC)'])/2
met['total_rain'] = (met['portRain(mm)']+met['stbdRain(mm)'])/2
met['wind_spd'] = (met['portTrueWindSpeed(knot)']+met['stbdTrueWindSpeed(knot)'])/2*1.944
met['wind_dir'] = (met['portTrueWindDir(degree)']+met['stbdTrueWindDir(degree)'])/2
met = met[['latitude(degree_north)','longitude(degree_east)','air_temp','atmPressure(mbar)','total_rain','wind_spd','wind_dir']]
met.rename(columns = {"latitude(degree_north)": "lat",
                            "longitude(degree_east)": "lon",
                            "atmPressure(mbar)": "air_pressure"}, inplace=True)

In [62]:
# convert observations to a dataset
cap2_ds = aer.to_xarray()
cap2_ds = cap2_ds.assign(met)

In [63]:
# remove outliers
cap2_ds = cap2_ds.where(cap2_ds['cn10_avg']<10000)

# Edit data attributes
cap2_ds.attrs['title'] = 'Aerosol and meteorology observations for the 2018 CAPRICORN II voyage.'
cap2_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [64]:
# add new source coordinate
cap2_ds = cap2_ds.assign_coords(Source='Cap2')

In [65]:
cap2_ds.load().to_netcdf(path=f_path+'cap18_obs.nc')

# MARCUS

In [66]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/AuroraAustralis/'

aer = pd.read_csv(f_path+'CN_MARCUS_1hr.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn_avg','cn_std','lat','lon']].dropna()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = pd.read_csv(f_path+'marcus-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [67]:
# convert observations to a dataset
aa_ds = aer.to_xarray()
aa_ds = aa_ds.assign(met)

In [68]:
# remove outliers
aa_ds = aa_ds.where(aa_ds['cn10_avg']<10000)

# Edit data attributes
aa_ds.attrs['title'] = 'Aerosol and meteorology observations for the 2017-2018 MARCUS voyage.'
aa_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [69]:
# add new source coordinate
aa_ds = aa_ds.assign_coords(Source='MARCUS')

In [70]:
aa_ds.load().to_netcdf(path=f_path+'aa1718_obs.nc')

# TAN1802

In [71]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/TAN1802/'
aer = xr.open_dataset(f_path+'CPC3010_TAN1802_2018_level_01.nc')
aer = aer.rename({'number_concentration_of_ambient_aerosol_particles_in_air':'cn10',
                  'number_concentration_of_ambient_aerosol_particles_in_air_flag':'cn10_flag'})
aer = aer.where(aer.cn10_flag == 1)
aer['cn10'] = aer['cn10']/1000000
aer = aer.resample(time='1D').mean()
aer = aer.rename({'cn10': 'cn10_avg'})

uw = keycutter(f_path+'weather.csv','Time (UTC)', 'Latitude', 'Longitude')


met = pd.read_csv(f_path+'TAN-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [72]:
# convert observations to a dataset
tan_ds = aer
tan_ds = tan_ds.assign(uw)
tan_ds = tan_ds.assign(met)

In [73]:
# remove outliers
tan_ds = tan_ds.where(tan_ds['cn10_avg']<10000)

# Edit data attributes
tan_ds.attrs['title'] = 'Aerosol and meteorology observations for the 2017-2018 TAN1802 voyage.'
tan_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [74]:
# add new source coordinate
tan_ds = tan_ds.assign_coords(Source='TAN1802')

In [75]:
tan_ds.load().to_netcdf(path=f_path+'tan1718_obs.nc')

# Cape Grim

In [76]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/CGO/'
aer = pd.read_excel(f_path+'cgbpaps_CN  2016-2018.xlsx', index_col='date')
aer = aer.astype(np.float64).resample('1D',kind='time').mean()
aer.index.name = 'time'
aer = aer[['cn3772_mean','cn3772_sd']]
aer.rename(columns={'cn3772_mean': 'cn10_avg', 'cn3772_sd': 'cn10_std'}, inplace=True)

met = pd.read_excel(f_path+'CG_met_2015-2018.xlsx', index_col='date')
met = met[['TempGC','Pressure','Rain','VWS10m','VWD10m']]
met.index.name = 'time'
met = met.resample('1D', kind='time').mean()
met.rename(columns={'TempGC': 'air_temp', 'Rain': 'total_rain', 'Pressure': 'air_pressure', 'VWD10m': 'wind_dir', 'VWS10m': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_08_25a_CGO_Radon_2015-2018_ForLiam.xlsx', sheet_name='2015-2018-RadonMetCO2', index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [77]:
# convert observations to a dataset
cgo_ds = aer.to_xarray()
cgo_ds = cgo_ds.assign(met)
cgo_ds = cgo_ds.assign(rdn)

In [78]:
# remove outliers
cgo_ds = cgo_ds.where(cgo_ds['cn10_avg']<10000)

# Edit data attributes
cgo_ds.attrs['title'] = 'Aerosol, meteorology and radon observations at the 2016-2018 Cape Grim Baseline Air Pollution Station.'
cgo_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [79]:
# add new source coordinate
cgo_ds = cgo_ds.assign_coords(Source='CGO')

In [80]:
cgo_ds.load().to_netcdf(path=f_path+'cgo1618_obs.nc')

# Macquarie Island

In [81]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/MQI/'
aer = pd.read_hdf(f_path+'ACRE_CN10_1H_FINAL.h5')
aer.index.name = 'time'
aer = aer[['cn_avg','cn_std']]
aer = aer.resample('1D', kind='time').mean()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = []
all_files = gb.glob(f_path+'AWS_hourly/*.csv')
met = pd.concat([pd.read_csv(v) for v in all_files])
met.index = met['MIN_OBSERVATION_DATE_'].astype('datetime64[ns]')
met.index.name = 'time'
met['AVG_RAINFALL'] = met['AVG_RAINFALL'].diff()
met['AVG_RAINFALL'][met['AVG_RAINFALL']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['AVG_RAINFALL'] = met_tmp['AVG_RAINFALL'].resample('1D', kind='time').sum()
met = met[['AVG_AIR_TEMPERATURE','AVG_WIND_DIRECTION','AVG_WIND_SPEED','AVG_AIR_PRESSURE','AVG_RAINFALL']]
met.rename(columns={'AVG_AIR_TEMPERATURE': 'air_temp', 'AVG_RAINFALL': 'total_rain', 'AVG_AIR_PRESSURE': 'air_pressure', 'AVG_WIND_DIRECTION': 'wind_dir', 'AVG_WIND_SPEED': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_09_15a_MQI_Radon_2013-2019_ForLiamLamprey.xlsx',
                    sheet_name='Radon_MQI_2013-2019',
                    index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [82]:
# convert observations to a dataset
mqi_ds = aer.to_xarray()
mqi_ds = mqi_ds.assign(met)
mqi_ds = mqi_ds.assign(rdn)

In [83]:
# remove outliers
mqi_ds = mqi_ds.where(mqi_ds['cn10_avg']<10000)

# Edit data attributes
mqi_ds.attrs['title'] = 'Aerosol, meteorology and radon observations at the 2016-2018 Macquarie Island Research Station.'
mqi_ds.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [84]:
# add new source coordinate
mqi_ds = mqi_ds.assign_coords(Source='MQI')

In [85]:
mqi_ds.load().to_netcdf(path=f_path+'mqi1618_obs.nc')

In [95]:
voyages = [rvi_ds,
           r2r_ds,
           i2e_ds,
           cap1_ds,
           cap2_ds,
           aa_ds,
           tan_ds,
           cgo_ds,
           mqi_ds,
]

# voyage_obs_ds = xr.concat(voyages, dim='Source')
# voyage_obs_ds

## Combine model data

In [87]:
# load in data
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/'
rvi_bx = xr.open_dataset(f_path+'RVInvestigator/rvi1619_bx400_track.nc').assign_coords(Source='RVI')

r2r_bx = xr.open_dataset(f_path+'Reef2Rain/r2r16_bx400_track.nc').assign_coords(Source='Reef2Rain')

i2e_bx = xr.open_dataset(f_path+'Ice2Equator/i2e16_bx400_track.nc').assign_coords(Source='Ice2Equ')

cap1_bx = xr.open_dataset(f_path+'Capricorn/cap16_bx400_track.nc').assign_coords(Source='Capricorn1')

cap2_bx = xr.open_dataset(f_path+'Capricorn/cap18_bx400_track.nc').assign_coords(Source='Capricorn2')

aa_bx = xr.open_dataset(f_path+'AuroraAustralis/aa1718_bx400_track.nc').assign_coords(Source='MARCUS')

tan_bx = xr.open_dataset(f_path+'TAN1802/tan1718_bx400_track.nc').assign_coords(Source='TAN1802')

cgo_bx = xr.open_dataset(f_path+'CGO/cgo1618_bx400_track.nc').assign_coords(Source='CGO')

mqi_bx = xr.open_dataset(f_path+'MQI/mqi1618_bx400_track.nc').assign_coords(Source='MQI')

bx400_data = [rvi_bx,
              r2r_bx,
              i2e_bx,
              cap1_bx,
              cap2_bx,
              aa_bx,
              tan_bx,
              cgo_bx,
              mqi_bx,]

bx400_ds = xr.concat(bx400_data, dim='Source')

In [88]:
# load in data
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/'
rvi_bx = xr.open_dataset(f_path+'RVInvestigator/rvi1619_cg283_track.nc').assign_coords(Source='RVI')

r2r_bx = xr.open_dataset(f_path+'Reef2Rain/r2r16_cg283_track.nc').assign_coords(Source='Reef2Rain')

i2e_bx = xr.open_dataset(f_path+'Ice2Equator/i2e16_cg283_track.nc').assign_coords(Source='Ice2Equ')

cap1_bx = xr.open_dataset(f_path+'Capricorn/cap16_cg283_track.nc').assign_coords(Source='Capricorn1')

cap2_bx = xr.open_dataset(f_path+'Capricorn/cap18_cg283_track.nc').assign_coords(Source='Capricorn2')

aa_bx = xr.open_dataset(f_path+'AuroraAustralis/aa1718_cg283_track.nc').assign_coords(Source='MARCUS')

tan_bx = xr.open_dataset(f_path+'TAN1802/tan1718_cg283_track.nc').assign_coords(Source='TAN1802')

cgo_bx = xr.open_dataset(f_path+'CGO/cgo1618_cg283_track.nc').assign_coords(Source='CGO')

mqi_bx = xr.open_dataset(f_path+'MQI/mqi1618_cg283_track.nc').assign_coords(Source='MQI')

cg283_data = [rvi_bx,
              r2r_bx,
              i2e_bx,
              cap1_bx,
              cap2_bx,
              aa_bx,
              tan_bx,
              cgo_bx,
              mqi_bx,]

cg283_ds = xr.concat(cg283_data, dim='Source')